In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

2023-03-22 01:03:40.581929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [13]:
def to_tensor(X_train, X_test, y_train, y_test):
    X_train = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
    X_test = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

    return X_train, X_test, y_train, y_test


In [14]:
X, y = df.copy().drop('Outcome', axis=1), df.Outcome
X_train, X_test, y_train, y_test = to_tensor(*train_test_split(X, y, test_size=0.3))

In [15]:
class NeuralNetwork(keras.Model):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = keras.Sequential([
            keras.layers.Dense(16, activation='relu', input_shape=(8,)),
            keras.layers.Dense(8, activation='relu'),
            keras.layers.Dense(4, activation='relu'),
            keras.layers.Dense(1, activation='sigmoid')
        ])

    def call(self, x):
        return self.linear_relu_stack(x)

In [16]:
model = NeuralNetwork()
print(model)

In [17]:
loss_fn = keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.RMSprop()

In [18]:
@tf.function
def train_step(X, y, model, loss_fn, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [19]:
@tf.function
def test_step(X, y, model, loss_fn):
    predictions = model(X)
    loss = loss_fn(y, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(predictions), y), tf.float32))
    return loss, accuracy

In [20]:
epochs = 1000
for t in range(epochs):
    train_loss = train_step(X_train, y_train, model, loss_fn, optimizer)
    test_loss, test_accuracy = test_step(X_test, y_test, model, loss_fn)

    if t % 100 == 0:
        print(f"Epoch {t+1}\n-------------------------------")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

print("Done!")

2023-03-22 01:04:45.672465: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-22 01:04:45.967952: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fb64c015470
2023-03-22 01:04:45.974978: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fb64c015470
2023-03-22 01:04:45.996931: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fb64c015470
2023-03-22 01:04:45.996970: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fb64c015470
2023-03-22 01:04:46.015688: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:4

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_7' defined at (most recent call last):
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/martinkatkovcin/anaconda3/envs/tensorflow/lib/python3.10/site-pac

In [1]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

: 

: 